## Make 3-d tensor with space(iris), app and time as dimension for median week traffic maps

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import time
import copy
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colrs

import pickle as pkl

from tqdm import tqdm
import torch
import gc


In [2]:
USER = os.getlogin()
WORKING_DIR = f'/home/{USER}/data/Land_use'
DATA_DIR = f'{WORKING_DIR}/data'
IMAGE_DIR = f'{WORKING_DIR}/images'
CITY_NAME = 'Paris'

In [3]:
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['CMU Serif Roman'] + plt.rcParams['font.serif']
plt.rcParams['font.size'] = 18

In [ ]:
# df_mask = pd.read_pickle(f'{DATA_DIR}/df_masks.pkl')
# city_row = df_mask[df_mask['name'] == CITY_NAME].iloc[0]

# city_shape = city_row['shape']
# city_mask = city_row['mask']
# city_left_x = city_row['left_x']
# city_bottom_y = city_row['bottom_y']

### Load Data

In [4]:
# filename = f'{DATA_DIR}/iris_traffic_maps_srca_median_week.pkl'
filename = '/home/jupyter-landy/data/Land_use/Santiago/iris_srca_traffic_maps_median_week.pkl'
df_traffic_maps = pd.read_pickle(filename)
df_traffic_maps.head()

,city,app,time,day_of_week,traffic_map_iris,iris_traffic_map_srca
0,Paris,Amazon Web Services,01:00:00,Friday,"[756557.75, 441333.875, 105561.203125, 211886....","[0.057484827291603036, 0.04971500651103322, 0...."
1,Paris,Amazon Web Services,16:00:00,Friday,"[2459350.0, 1037933.375, 325833.90625, 1299623...","[-0.11984061817532497, -0.04115146238746292, 0..."
2,Paris,Apple App Store,17:30:00,Thursday,"[187772864.0, 43094516.0, 8510793.0, 65463148....","[0.168997213096074, -0.140662831596498, -0.178..."
3,Paris,Apple Mail,03:30:00,Wednesday,"[8740.5126953125, 504.6463928222656, 1567.0026...","[-0.2957118163870927, -0.9230939313388649, -0...."
4,Paris,Apple Music,06:30:00,Thursday,"[1702352.75, 1864531.75, 276829.8125, 2417798....","[-0.24182257603396887, 0.006167592594721744, -..."


In [5]:
df_traffic_maps = df_traffic_maps[(df_traffic_maps['time'] != '00:00:00') & (df_traffic_maps['time'] != '00:30:00')]
df_traffic_maps = df_traffic_maps.reset_index(drop=True)
df_traffic_maps.shape

(21896, 6)

### Adding a unique time field by combining day_of_week and time

In [6]:
day_list = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
df_traffic_maps['day_time'] = df_traffic_maps.apply(lambda row: f'{day_list.index(row["day_of_week"])}_{row["time"]}', axis=1)
df_traffic_maps.head(2)

,city,app,time,day_of_week,traffic_map_iris,iris_traffic_map_srca,day_time
0,Paris,Amazon Web Services,01:00:00,Friday,"[756557.75, 441333.875, 105561.203125, 211886....","[0.057484827291603036, 0.04971500651103322, 0....",5_01:00:00
1,Paris,Amazon Web Services,16:00:00,Friday,"[2459350.0, 1037933.375, 325833.90625, 1299623...","[-0.11984061817532497, -0.04115146238746292, 0...",5_16:00:00


### Removing undesired apps

In [7]:
apps_to_del = [
    'Apple App Store',
    'Apple Siri',
    'Apple Web Services',
    'Apple iCloud',
    'Fornite',
    'Microsoft Skydrive',
    'Microsoft Store',
    'Microsoft Web Services',
    'Molotov TV',
    'Tor',
    'Web Adults',
]

### Make Tensor

In [8]:
day_times = sorted(list(set(df_traffic_maps['day_time'])))
apps = list(set(df_traffic_maps['app']))
apps = sorted(list(set(apps) - set(apps_to_del)))

day_times_dim = len(day_times)
app_dim = len(apps)
space_dim = len(df_traffic_maps['iris_traffic_map_srca'].iloc[0])

day_times_idx = {t: i for i, t in enumerate(day_times)}
apps_idx = {a: i for i, a in enumerate(apps)}

tensor = np.zeros((space_dim, app_dim, day_times_dim))
print(tensor.shape)

(2800, 58, 322)


In [9]:
#apps

In [10]:
for row in tqdm(df_traffic_maps.to_dict('records')):
    app = row['app']
    if app in apps_to_del:
        continue
    day_time = row['day_time']
    space = row['iris_traffic_map_srca'].copy()
    # space = space[city_mask == 1]

    app_idx = apps_idx[app]
    time_idx = day_times_idx[day_time]
    tensor[:, app_idx, time_idx] = space

100%|██████████| 21896/21896 [00:05<00:00, 4072.84it/s]


### Save Tensor

In [ ]:
fd = open(f'{DATA_DIR}/iris_tensor_rm_midnight.pkl', 'wb')
pkl.dump([tensor], fd)
fd.close()